In [ ]:
import os
import main_fte

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
# Configurations for the optimisation.
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos")
out_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/analysed")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "zico", "cheetah_videos")
# out_dir = os.path.join("/", "data", "zico", "analysed")
data_path = "kinetic_tests/2009_06_18/trail02/run"
start_frame = 351
end_frame = 750
dlc_thresh = 0.5

In [ ]:
# Run the optimisation.
main_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, sync_offset=[{ "cam": 2, "frame": 259}], out_dir_prefix=out_dir, generate_reprojection_videos=False)

In [ ]:
# CAM3 R1: [[0.4957129000771898, 0.5253279996923343, -0.6915917968291762], [0.8418697862836664, -0.48628999092733044, 0.23404552477632265], [-0.21336350126765824, -0.6982496240245583, -0.6833179925133946]]\nCAM3 t1: [-4.947536016971906, -0.35895328179391334, 1.7498181973450575]\nCAM3 R2: [[0.5105622975910866, 0.4678771521538111, -0.7213994114115613], [0.8379710215728567, -0.4587741387598743, 0.2955179463146039], [-0.19269329854972395, -0.7553921234168376, -0.6263002735111426]]\nCAM3 t2: [-1.8917734920094507, -0.18678344856027781, 2.263638758427488]\nCAM3 R3: [[-0.3640528295521911, 0.5006497318461484, 0.7853759502904673], [-0.8738433439609085, -0.4753836667060206, -0.10202048669223136], [0.32227836959285194, -0.7234363935134697, 0.6105542047960856]]\nCAM3 t3: [2.812922557158232, 0.42021398324636544, 12.101725298421254]
# CAM3 R3: [[-0.9923299638742579, -0.016863295400034794, -0.12246176348365519], [0.12016950135236125, 0.10074563445557491, -0.9876282797445483], [0.028992157177546676, -0.9947692933377399, -0.09794645969732849]]\nCAM3 t3: [-2.8990393758446156, 0.499999702974232, 14.201723238196715]
# CAM3 R3: [[-0.993882102201052, -0.015203200638411865, -0.10939498855225296], [0.10729099186062215, 0.10216272581603371, -0.9889648393538173], [0.026211522666597268, -0.9946515170567514, -0.09990654054425528]]\nCAM3 t3: [-2.853796422645351, 0.49999990006064127, 14.289819435221103]
# Plot the 3D trajectory for visual inspection.
main_fte.plot_cheetah(root_dir, data_path, fte_type="fte", out_dir_prefix=out_dir)

In [ ]:
# Generate metrics.
main_fte.metrics(root_dir, data_path, start_frame, end_frame, dlc_thresh)